## Introduction to SCEC Ground Motion Simulation Validation (GMSV) Toolkit

In this notebook we present a brief introduction to SCEC GMSV Toolkit and use it to compare the pseudospectral accelerations (PSAs) of ground motion generated in the Broadband Platform (BBP) against recorded ground motions for the 1994 Northridge earthquake. We also include a hands-on exercise where users can use the GMSV Toolkit seismogram comparison plotting tools to visualize the effects of Broadband Platform's site response module.

SCEC's goal with GMSV Toolkit is to make existing, well verified, and useful software available to a broad seismological and engineering community of researchers interested in ground motion simulations validation. SCEC's GMSV Toolkit leverages over a decade of scientific, engineering, and software development work completed by dozens of SCEC contributors, including the groups working with the Broadband Platform, the GMSV TAG, the High-F group, among others.

The GMSV codes compute various ground motion metrics, compare them with empirical data and models, and deliver suites of statistical products, including plots, to facilitate the evaluation. It aggregates methods into a standalone package so that their full capabilities are accessible for the validation of any simulation set.

### Overview

1. Getting started
2. Broadband Platform background
    1. Validation simulation inputs
    2. Validation simulation outputs
    3. Plotting map with stations and rupture
3. Looking at simulated timeseries
4. Calculating PSA
    1. Computing PSA for simulated data
    2. Computing PSA for observed data
5. Comparing PSA: simulations against observations
6. Generating PSA GoF Plot
    1. Calculating residuals
    2. PSA GoF
    3. Other GoFs
7. Class exercise
    1. Selecting sites for comparison
    2. Getting started
    3. Selecting input and output files
    4. Comparing stations with lower Vs30 value
    5. Comparing stations with higher Vs30 value
8. Further reading
9. Acknowledgements

### 1. Getting started

In this notebook we walk users through the steps involved in the GMSV Toolkit's PSA processing pipeline. Several of the codes used in this notebook have been used in the post-processing stages of the Broadband Platform for more than ten years, and have been migrated to the GMSV Toolkit repository as a standalone package. This enables users to take advantage of these codes outside of the Broadband Platform.

The codes included in the GMSV Toolkit package are designed to work independently and can be used separately. Each module includes a command-line interface that allows it to be used from the command-line or automated with a script. Additionally, users can import each module into their Python programs and use the tools directly by calling the function they need. In this notebook we will focus on the using the codes via the Python interface.

Briefly, in the PSA GoF pipeline, acceleration timeseries go through the RotDXX module where RotD50 is computed at 63 periods ranging from 0.01s to 10s. The output can be compared station by station with the plot_rotdxx module. These RotD50 files can also be aggregated across all stations and compared against each other with the PSA GoF tool. Finally, the PSA GoF Plot module can be used to generate different PSA GoF plots so users can see how the two datasets match. Please note that the pipeline above can work with simulations versus observed data, or with two sets of simulated data.

The diagram below illustrates the processing steps involved in the PSA processing pipeline:

<img align="center" width="800" src="images/gmsvtoolkit_psa_pipeline.png">

We will start our processing with the outputs provided by the Platform, and use different tools from the GMSV Toolkit to obtain a PSA Goodness-of-Fit (GoF) plot that can be used to visualize how the simulation results from the Platform compare against recorded timeseries from the 1994 Northridge earthquake.

For this exercise, we will follow the PSA pipeline described above. We will use the RotDXX module to compute PSA for both calculated and observed timeseries. We will generate comparison plots of observed versus calculated timeseries for a station to check how they compare at different periods. Next, we will combine the PSA data from all stations and generate residuals that will be used in the last step to generate Goodness-of-Fit (GoF) plots.

Now, before we get started with this notebook, we will import a couple of basic modules that will provide useful functions for using paths and showing results later on.

In [ ]:
from IPython.display import Image, display
import os

### 2. Broadband Platform background

The Broadband Platform (BBP) is a software system that can generate 0-20+ Hz seismograms for historical and scenario earthquakes in California, Eastern North America, and Japan using several alternative computational methods. In a validation simulation, we use a historical event and select station locations for which we have recorded data available. We can then compare the synthetic seismograms produced by the Platform against the recorded data and see how simulation methods match historical events. The Broadband Platform can also run scenario simulations, where it can generate ground motions for an arbitrary event at user-selected locations.

In this notebook, we will be using data from a Broadband Platform validation simulation that compares how a simulation method matches data recorded from the 1994 Northridge earthquake in Southern California.

#### A. Validation simulation inputs

<img align="right" width="350" src="images/labasin500.png">

Running the Broadband Platform involves selecting a simulation method and a simulation region that provides a 1-D velocity model that will be used to compute the synthetic seismograms. Although these items are not used in this notebook, we wanted to mention them for completeness. On the right, we show the 1-D velocity profile used for the LA Basin region, with a Vs30 set to 500m/s. After seismogram synthesis is completed, the Broadband Platform will run a site amplification module that will correct the simulated data at each station to the Vs30s provided in the station list (see below).

Also needed for a validation simulation is a description of the earthquake. The Broadband Platform uses a simple source description file (also called a SRC file with a ".src" extension). This file includes the fault geometry, location, mechanism, and other basic parameters used in the simulation. Users will also need to specify a list of stations for which we would like to compute synthetic seismograms. The station list contains information for each station, including station name, coordinates (latitude and longitude), Vs30, and also a range of frequencies for which recorded data is available.

Finally, for a validation simulation will need a set or recorded seismograms, one for each station, containing a three-component acceleration timeseries. These observations can then be compared against the simulated timeseries produced by the Platform using a variety of metrics.

Below we show examples of both a source description file and a station list, which will be used later in this notebook. The Northridge source description file includes magnitude, fault geometry and location, and other simulation parameters needed by the various Broadband Platform simulation methods. The station file contains the station location (longitude and latitude), a station name or identifier, Vs30 for the site, and the range of frequencies for which recorded data is valid for this station.

#### B. Validation simulation outputs

After the seismogram synthesis part of the Broadband Platform completes, we will have one velocity and one acceleration timeseries file for each station in our station list. These files, in BBP format, contain four columns, with time (in seconds), and two horizontal and one vertical components (in cm/s or cm/s/s). An example is provided below:

Please note that these are broadband seismograms, which combine both low- and high-frequency seismogram generation techniques from different Broadband Platform components. If the site response module was selected, these output timeseries have already been corrected to the Vs30 of each station in the station list.

#### C. Plotting map with stations and rupture

The data package provided with this notebook contains a full set of Broadband Platform seismograms for the 1994 Northridge earthquake. The set includes the Northridge source description file, a station list with 39 stations, and a set of observation files. We can set up paths for these files using the commands below:

In [ ]:
# First we configure the location of BBP input files
base_dir = "/home/scecuser/gmsvtoolkit_tutorials"
gmsv_data_package_dir = os.path.join(base_dir, "tutorial_data")
bbp_inputs = os.path.join(gmsv_data_package_dir, "bbp_inputs")

# Set up station list and source description files
station_file_3_stations = os.path.join(bbp_inputs, "nr_v19_06_2_3_stations.stl")
station_file_all_stations = os.path.join(bbp_inputs, "nr_v19_06_2.stl")
src_file = os.path.join(bbp_inputs, "nr_v20_07_1.src")

In [ ]:
# Set input folders for simulation and observation inputs

# These two folders also contain some preprocessed data to speed up
# some of the computations below, users can regenerate them using
# the tools available in this notebook
input_dir_sims = os.path.join(gmsv_data_package_dir, "bbp_results")
input_dir_obs = os.path.join(gmsv_data_package_dir, "obs_data")

# Set output folders where we want to store our results
output_dir_sims = os.path.join(gmsv_data_package_dir, "sims_processed")
output_dir_obs = os.path.join(gmsv_data_package_dir, "obs_processed")
output_dir = os.path.join(gmsv_data_package_dir, "gmsv_toolkit_psa_pipeline")

In order to check the source description file and the station list, we will use the plot_map module in the plots package to create a map with the Northridge rupture and the list of stations used by the Broadband Platform. We will first import the plot_map module from the plots package:

In [ ]:
from plots.plot_map import plot_map

Next, we set a plot title, an output file, and invoke the plot_map method with the station list and source description file:

In [ ]:
plot_title = "Northridge Fault Trace with all Stations"
all_stations_map = "all_stations_map.png"
plot_map(station_file_all_stations, src_file, plot_title, output_dir, output_file=all_stations_map)

We can also generate a second map with a smaller set of stations that we will use for some of the examples below. We just substitute the station file and provide a different output file:

In [ ]:
plot_title = "Northridge Fault Trace with 3 Stations"
three_stations_map = "three_stations_map.png"
plot_map(station_file_3_stations, src_file, plot_title, output_dir, output_file=three_stations_map)

To visualize the two station maps we just generated, we can use the display function from Jupyter.

In [ ]:
for filename in [os.path.join(output_dir, all_stations_map), os.path.join(output_dir, three_stations_map)]:
    display(Image(filename))

### 3. Looking at simulated timeseries

Before we run the PSA pipeline, we will plot a set of seismograms for one of the stations used in this notebook. The plotting tool we will use can generate 3-component plots for acceleration, velocity, and displacement so the first step will be to use the gmsv_tools module from the utils package to convert BBP acceleration files into displacement and velocity. We can do this by importing the module and then calling the convert_station_file function:

In [ ]:
from utils import gmsv_tools

In [ ]:
# Create acceleration, velocity, and displacement timeseries for simulated data
for output_format in ['acc', 'vel', 'dis']:
    gmsv_tools.convert_station_file(station_file_3_stations, input_format="acc", output_format=output_format,
                                input_dir=input_dir_sims, output_dir=output_dir_sims)

We repeat the step above for the observation files so that we can compare the two later:

In [ ]:
# Create acceleration, velocity, and displacement timeseries for observed data
for output_format in ['acc', 'vel', 'dis']:
    gmsv_tools.convert_station_file(station_file_3_stations, input_format="acc", output_format=output_format,
                                input_dir=input_dir_obs, output_dir=output_dir_obs, input_suffix=".bbp")

Now that we have generated two sets of seismograms, we can use the PlotSeismograms module from the plots package to generate a comparison plot. We set basic parameters like plot title, xmin, xmax, and two arrays with the files to plot and labels we would like to use:

In [ ]:
from plots.plot_seismograms import PlotSeismograms

# Create plotting object, we want to plot 3 components
plot_seism_obj = PlotSeismograms(n_comp=3)

# Pick station
station_name = "2001-SCE"

# Two input files, observed and calculated seismograms
obs_file = os.path.join(output_dir_obs, "%s.acc.bbp" % (station_name))
sim_file = os.path.join(output_dir_sims, "2354660.%s.acc.bbp" % (station_name))

# Create array with input files, and another array with labels to use for the plot
input_files = [sim_file, obs_file]
labels = ["Sim", "Obs"]

# Select plotting range 0-40 seconds
xmin = 0.0
xmax = 40.0

# Pick a plot title and set the output file
plot_title = "Seismogram comparison for station %s"  % (station_name)
timeseries_plot = os.path.join(output_dir, "2354660.%s_seis.png" % (station_name))

# Generate plot
plot_seism_obj.run_single_station(input_files, labels,
                                  timeseries_plot, station_name,
                                  xmin, xmax, plot_title=plot_title)

We can see the plot generated with the command below:

In [ ]:
display(Image(timeseries_plot))

In the plots above we can see acceleration, velocity, and displacement plots for both simulated and observed timeseries. We can observe that simulated data begins at origin time and is therefore shown after the observed data.

### 4. Calculating PSA

For this step, we will compute PSA values for both simulated and observed data. We will be using GMSV Toolkit's RotdXX module, which can be used to read an acceleration time series and compute RotD50 values for a set of 63 periods ranging from 0.01s to 10s.

We will import the RotDxx module from the metrics package and initialize the module:

In [ ]:
from metrics import rotdxx
rotdxx_obj = rotdxx.RotDXX(mode="rotd50")

#### A. Computing PSA for simulated data

The required inputs for this module include a list of stations and input acceleration timeseries for each station. For this example, we will use a station list with only three stations. The RotDXX module expects timeseries for all stations to be in a single folder, and that all input files include the station name and feature a ".acc.bbp" extension. Additionally, we will also provide an output folder where we would like to store the computed PSA for each station.

With all input files in place, the next step is to run the RotDXX code, which will compute PSA for each of the stations included in the station list. As with most modules in the GMSV Toolkit software, the RotDXX module can also be used with a single station, where only an input and output files need to be specified.

In [ ]:
rotdxx_obj.run_station_mode(station_file_3_stations, input_dir_sims, output_dir_sims)

#### B. Computing PSA for observed data

In order to calculate PSA values for the observed data we will run the RotDXX code a second. Please note that we will are using the same station list we used in the previous step, and just changed the input and output folders so that we process the observed files:

In [ ]:
rotdxx_obj.run_station_mode(station_file_3_stations, input_dir_obs, output_dir_obs, input_suffix=".bbp")

### 5. Comparing PSA: simulations against observations

Now that we have PSA files for both simulated and observed data, we can create a plot comparing them using the plot_rotdxx module. We import the plot_rotdxx module from the plots package and initialize it:

In [ ]:
from plots import plot_rotdxx
plot_rotdxx_obj = plot_rotdxx.PlotRotDXX(mode="rotd50")

We are going to use the same station list and we just create an "input_dirs" array containing a list of folders where our data is. Additionally, we create a "labels" array containing labels for each of the data sets we specified in the "input_dirs" array. For our example, we will have two labels, "NR" refers to the observed data from the Northridge event, and we will use the Broadband Platform simulation id (2354660) for the simulated data. Setting up comp_label is optional, and allows us to specify a prefix to the output png file. 

In [ ]:
input_dirs = [output_dir_obs, output_dir_sims]
labels = ["NR", "2354660"]
comp_label = "NR_2354660"

Once we have everything set up, we can just call the run_station_mode method and we should get a comparison plot for each station in our station list.

In [ ]:
plot_rotdxx_obj.run_station_mode(station_file_3_stations, input_dirs, labels, output_dir, comp_label)

Finally, we can see the end comparison for station 2001-SCE using the following command:

In [ ]:
display(Image(os.path.join(output_dir, "NR_2354660_2001-SCE_rotd50.png")))

In the plots above we can see PSA values for the two horizontal components and RotD50 (right). The green line shows calculated data and the blue line shows PSA values for the recorded seismogram. The two vertical lines (purple and red) come from the station list and indicates limits for which the recorded data is valid for this particular station.

### 6. Generating PSA GoF plot

Now that we have computed PSA for both simulations and observed data, we can generate a Goodness-of-Fit (GoF) plot using the two datasets. Although we can generate a GoF plot with the 3 stations that we have, it will look better with more stations so that's what we will use. The training data package includes pre-computed PSA files for all stations (both simulated and observed data), but users are welcome to replace the short station list with the full station list in the examples above and regenerate the files themselves, it will just take a little longer.

#### A. Calculating residuals

First we import the psa_gof module from the stats package. Then we can use the max_cutoff parameter to exclude stations that are too far away from the rupture. In our example, we set it to 120km to include all stations in our list:

In [ ]:
from stats import psa_gof
max_cutoff = 120

Next, we instantiate our PSAGoF module and run it to generate the residual files for all stations:

In [ ]:
psa_gof_obj = psa_gof.PSAGoF(mode="rotd50", max_cutoff=max_cutoff, comp_label=comp_label)
psa_gof_obj.run_psa_gof(station_file_all_stations, src_file, input_dir_obs, input_dir_sims, output_dir)

#### B.  Plotting GoF

Now that we have calculated the residuals using the psa_gof module, we can generate a GoF plot using the plot_psa_gof method available in the plots package. We import plot_psa_gof, set up a plot_title for our plot, and generate the PSA GoF using:

In [ ]:
from plots.plot_psa_gof import plot_psa_gof
plot_title = "GoF Comparison between NR and simulation 2354660"
plot_psa_gof(output_dir, output_dir, plot_title, comp_label, max_cutoff=max_cutoff)

Finally, we can visualize the generated GoF plot using the command below:

In [ ]:
display(Image(os.path.join(output_dir, "gof-NR_2354660_r0-120-rd50.png")))

For the plot above we will focus on the first subplot, which shows the RotD50 values. This plot shows how simulated data compare the recorded data at different periods. The red line shows the mean and reflects how well the two sets match. Note that for periods between 1s and 5s the red line is close to zero, indicating the two sets are pretty close. For periods < 1.0s (higher frequencies) the red line dips below zero, signaling the simulated data is overpredicting the recorded data.

#### C. Other GoFs

In addition to the PSA GoF plot by periods, the GMSV Toolkit package is able to generate PSA GoFs by distance, Vs30, and on a map. These plots compare calculated data against recorded data and use the same data set as the PSA GoF above. We can import the needed modules from the plots package and generate the corresponding plots using the following commands:

In [ ]:
from plots.plot_vs30_gof import plot_vs30_gof
from plots.plot_dist_gof import plot_dist_gof
from plots.plot_map_gof import plot_map_gof
plot_mode = "rd50"

# Generate GoFs by Vs30, distance, and on a map
plot_vs30_gof(output_dir, output_dir, comp_label, plot_mode, plot_title=plot_title)
plot_dist_gof(output_dir, output_dir, comp_label, plot_mode, plot_title=plot_title)
plot_map_gof(output_dir, output_dir, comp_label, plot_mode, src_file, station_file_all_stations, plot_title=plot_title)

The first plot we generate is the Vs30 GoF plot, which can be used to check for any bias related to site conditions (in this case the Vs30 value for each station). We can see the plot using the following command:

In [ ]:
display(Image(os.path.join(output_dir, "gof-vs30-NR_2354660-rotd50.png")))

It is also useful to visualize the data per station on a map. This allows us to check for bias related to station location/directivity effects. Just as with the main PSA GoF plot, positive values (blue) indicate simulations are underprediting the recorded data while negative values (red) show stations in which the synthetic seismograms are overpredicting the data:

In [ ]:
display(Image(os.path.join(output_dir, "gof-map-NR_2354660-rotd50.png")))

Finally, we can also plot the same data per distance and check for any biases/trends. Similar to the plots above, we show data for each station using 8 different periods:

In [ ]:
for filename in [os.path.join(output_dir, "gof-dist-linear-NR_2354660-rotd50.png"),
                 os.path.join(output_dir, "gof-dist-log-NR_2354660-rotd50.png")]:
    display(Image(filename))

### 7. Class exercise

In this class exercise, we will use GMSV Toolkit's seismogram plotting tools to compare two datasets that come from the Broadband Platform. The first dataset is the Northridge simulation we have used in the first part of this notebook. The second dataset uses the same source description and station list but does not include the use of BBP's site response module. Therefore, all seismograms on the second dataset are calculated with the Vs30 value used when generating the 1-D greens functions, which was set to 500m/s.

#### A. Selecting sites for comparison

When running the site response module, we expect to see an increase in ground motions for sites for which Vs30 is less than 500m/s, and a decrease in sites with a Vs30 greater than 500m/s. Looking at the complete Northridge station list below:

For our comparison plots, we will try to pick two sites from the list above: one with a lower Vs30 and one with a higher Vs30 value. For a lower Vs30 site, we can pick a site like station 2029-CAS with a Vs30 value of 266m/s. For a higher Vs30 location, we can select station 2012-WON, with a Vs30 value of 1222m/s. These two sites should produce good comparison plots for comparing simulations with and without the Broadband Platform site response module.

#### B. Getting started

The first step for our comparison is to import needed Jupyter modules (if not done already), and import the PlotSeismograms module from GMSV Toolkit's plot package. Also, we can instantiate the plot seismograms class and set it to generate three-component velocity comparisons:

In [ ]:
# Import Jupyter modules so we can see the plots
from IPython.display import Image, display
import os

# Import PlotSeismograms module from the plots package
from plots.plot_seismograms import PlotSeismograms

# We would like to see only velocity seismograms
plot_seism_obj = PlotSeismograms(mode=["vel"], n_comp=3)

#### C. Selecting input and output files

The two Broadband Platform datasets included in this tutorial exercise data package can be found in the "bbp_results", and "bbp_results_no_site" folders. We can set up all required input and output folder with the following commands:

In [ ]:
base_dir = "/home/scecuser/gmsvtoolkit_tutorials"
gmsv_data_package_dir = os.path.join(base_dir, "tutorial_data")
input_dir_sims_site = os.path.join(gmsv_data_package_dir, "bbp_results")
input_dir_sims_nosite = os.path.join(gmsv_data_package_dir, "bbp_results_no_site")
output_dir = os.path.join(gmsv_data_package_dir, "gmsv_toolkit_psa_pipeline")

#### D. Comparing stations with lower Vs30 value

We can start by configuring the station name we would like to use for our comparison. As suggested above, we can use station "2029-CAS", but other stations with a lower Vs30 should also work for this example. Following this, we can create two arrays containing the two input files, and a corresponding array with the labels we would like to use in our plots. We can try to run the comparison using the other default parameters as configured below, but we can always come back and adjust xmin, xmax, and set up a different plot title if we want to customize things a bit.

In [ ]:
# Set station name and input files
station_name =

sim_site_file = os.path.join(input_dir_sims_site, "2354660.%s.vel.bbp" % (station_name))
sim_nosite_file = os.path.join(input_dir_sims_nosite, "2359040.%s.vel.bbp" % (station_name))

# Set up arrays of input files and labels
input_files =
labels =

# Limits for the plots
xmin = 0.0
xmax = 100.0

# Plot title and output filename
plot_title = "Seismogram comparison"
comp_low_vs30_plot = os.path.join(output_dir, "seismogram_comparison.%s_seis.png" % (station_name))

# Create plot
plot_seism_obj.run_single_station(input_files, labels,
                                  comp_low_vs30_plot, station_name,
                                  xmin, xmax, plot_title=plot_title)

Finally, we can display our generated plot with the command below:

In [ ]:
display(Image(comp_low_vs30_plot))

#### E. Comparing stations with higher Vs30 value

Next, we would like to follow the same procedure once again, but this time using a station with a higher Vs30 value. We can use station "2012-WON" as suggested above, or a different one that also features a higher Vs30.

In [ ]:
# Set station name and input files
station_name = 
sim_site_file = os.path.join(input_dir_sims_site, "2354660.%s.vel.bbp" % (station_name))
sim_nosite_file = os.path.join(input_dir_sims_nosite, "2359040.%s.vel.bbp" % (station_name))

# Set up arrays of input files and labels
input_files = 
labels = 

# Limits for the plots
xmin = 0.0
xmax = 100.0

# Plot title and output filename
plot_title = "Seismogram comparison"
comp_high_vs30_plot = os.path.join(output_dir, "seismogram_comparison.%s_seis.png" % (station_name))

# Create plot
plot_seism_obj.run_single_station(input_files, labels,
                                  comp_high_vs30_plot, station_name,
                                  xmin, xmax, plot_title=plot_title)

Finally, we can display our generated plot with the command below:

In [ ]:
display(Image(comp_high_vs30_plot))

### 8. Further reading

#### Software reference

The primary reference for the Broadband Platform software system (v15.3.0 and later) is:

* Maechling, P. J., F. Silva, S. Callaghan, and T. H. Jordan (2015). SCEC Broadband Platform: System Architecture and Software Implementation, Seismol. Res. Lett., 86, no. 1, doi: 10.1785/0220140125

#### Ground Motion Simulation Validation Method Reference
The primary reference that describes the validation process developed by the Broadband Platform group to establish that the BBP platform software produces results are suitable for use in engineering applications is:

* Goulet, C.A., Abrahamson, N.A., Somerville, P.G. and K, E. Wooddell (2015) The SCEC Broadband Platform Validation Exercise: Methodology for Code Validation in the Context of Seismic-Hazard Analyses, Seismol. Res. Lett., 86, no. 1, doi: 10.1785/0220140104

#### GitHub repo

Software related to this notebook can be found on GitHub:

* GMSV Toolkit: https://github.com/SCECcode/gmsvtoolkit

* GMSV Toolkit Tutorial: https://github.com/SCECcode/gmsvtoolkit_tutorials

* SCEC Broadband Platform: https://github.com/SCECcode/bbp

### 9. Acknowledgements

The following developers have contributed to the development of the GMSV Toolkit software package.

* Norm Abrahamson, Pacific Gas & Electric (PG&E)
* John Anderson, University of Nevada, Reno (UNR)
* Ralph Archuleta, University of California Santa Barbara (UCSB)
* Domniki Asimaki, California Institute of Technology (Caltech)
* Karen Assatourians, Western University, Canada
* Gail Atkinson, Western University, Canada
* Shima Azizzadeh-Roodpish, University of Memphis
* Jeff Bayless, AECOM
* Alexander Nikolas Breuer, Friedrich Schiller University Jena
* Scott Callaghan, Southern California Earthquake Center (SCEC)
* Jorge Crempien, Pontificia Universidad Católica de Chile
* Christine Goulet, Southern California Earthquake Center (SCEC)
* Robert Graves, United States Geological Survey (USGS)
* Behzad Hassani, British Columbia Hydro and Power Authority (BC Hydro)
* Asako Iwaki, National Research Institute for Earth Science and Disaster Resilience (NIED)
* Chen Ji, University of California Santa Barbara (UCSB)
* Thomas H. Jordan, Southern California Earthquake Center (SCEC)
* Naeem Khoshnevis, University of Memphis
* Philip Maechling, Southern California Earthquake Center (SCEC)
* Hiroe Miyake, Earthquake Research Institute, University of Tokyo
* Kim Olsen, San Diego State University (SDSU)
* Arben Pitarka, Lawrence Livermore National Laboratory (LLNL)
* Sanaz Rezaeian, United States Geological Survey (USGS)
* William Savran, San Diego State University (SDSU)
* Jian Shi, California Institute of Technology (Caltech)
* Fabio Silva, Southern California Earthquake Center (SCEC)
* Andreas Skarlatoudis, AECOM
* Patrick Small, Southern California Earthquake Center (SCEC)
* Paul Somerville, AECOM
* Seok-Goo Song, Korea Institute of Geoscience and Mineral Resources (KIGAM)
* Ricardo Taborda, Universidad EAFIT
* Rumi Takedatsu, San Diego State University (SDSU)
* Nan Wang, San Diego State University (SDSU)
* Ke Xu, San Diego State University (SDSU)